<a href="https://colab.research.google.com/github/submarinejuice/CP322-Final-Project-Group-9/blob/main/cp322_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multimodal Physiological Representation Learning for Predicting Risky Financial Decisions



**Research Question** - Can we predict whether a participant will invest in a risky asset on a given trial from:
Market context (expected return, volatility)
Physiological arousal (aSCRs)

**Secondary Research Question:**
Do we see comparable physiological signatures of stress/arousal in a real-world wearable dataset (WESAD), and can we learn a shared representation of physiological state that transfers across tasks?

**Motivation** - real financial decisions are emotional

**Problem** - predicting investment choice from market + physio data



#1. Setup & Reproducibility

## 1.1 Clone & Pull from our Repository

In [ ]:

import os
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")  # optional, makes plots nicer


REPO_URL = "https://github.com/submarinejuice/CP322-Final-Project-Group-9"
REPO_NAME = "CP322-Final-Project-Group-9"

if not os.path.exists(REPO_NAME):
    # First time in this Colab session: clone the repo
    !git clone {REPO_URL}
else:
    # Repo already there in this runtime: pull latest changes
    %cd {REPO_NAME}
    !git pull
    %cd /content

# Move into repo so relative paths work
%cd /content/{REPO_NAME}


## 1.2 Kaggle Setup to pull WESAD dataset instead of downloading the data

Must use your own Kaggle API as documented in the README file. Use your own kaggle.JSON file through the upload prompt that appears when this cell is run.

In [ ]:
!pip install kaggle

import os
import shutil
from google.colab import files

print("CWD:", os.getcwd())
os.makedirs('/content/.kaggle', exist_ok=True)
os.makedirs('data', exist_ok=True)

# Upload from laptop
uploaded = files.upload()   # select your kaggle.json / kaggle.JSON
fname = list(uploaded.keys())[0]
print("Uploaded:", fname)

# File is saved in the *current* directory, so src is just fname
src = fname
dst = '/content/.kaggle/kaggle.json'

shutil.move(src, dst)

# Fix permissions and inspect
!chmod 600 /content/.kaggle/kaggle.json



##1.2.5 Use Google Drive to temporarily store data so you don't have to rerun the download commands for such a large dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Allow notebook to temporarily store WESAD dataset with your drive

Downloading WESAD Dataset once the above has been complete.

# 1.3 WESAD download
 Will skip downloads if you already have the files in your drive.

In [ ]:
# Ensure Kaggle sees the right config
import os, shutil

print("Using config from /content/.kaggle/kaggle.json")

os.environ["KAGGLE_CONFIG_DIR"] = "/content/.kaggle"

!mkdir -p ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

print("kaggle.json copied to ~/.kaggle (not printed for security).")


adding an area that will say whether or not you need to redownload the dataset so that you dont double download accidentally !

In [ ]:
import os

# Path to WESAD after unzipping
WESAD_DIR = "data/WESAD"

if os.path.exists(WESAD_DIR) and len(os.listdir(WESAD_DIR)) > 0:
    print("✔ WESAD dataset already exists. Skipping download.")
else:
    print("⬇ Downloading WESAD dataset from Kaggle...")
    !kaggle datasets download -d orvile/wesad-wearable-stress-affect-detection-dataset -p data/
    !unzip -o "data/*.zip" -d data/
    print("✔ Download complete!")

if not os.path.exists("/content/.kaggle/kaggle.json"):
    raise FileNotFoundError(
        "kaggle.json missing — upload via files.upload() before continuing."
)



# 1.3.5 Implementing a loader file

In [ ]:
%%writefile wesad_loader.py
import os
from typing import Dict, Any, List

import numpy as np
import pandas as pd
from pandas.errors import EmptyDataError, ParserError

WESAD_PATH = "data/WESAD"
E4_FILES = ["ACC", "BVP", "EDA", "HR", "IBI", "TEMP", "tags"]


def _read_signal_csv(path: str) -> np.ndarray:
    """
    Read a 1-column CSV into a 1D numpy array.
    If the file is empty or malformed, return an empty array instead of crashing.
    """
    print(f"[wesad_loader] reading {path}")
    try:
        # Quick check by file size
        if os.path.getsize(path) == 0:
            print(f"[wesad_loader] WARNING: {path} is 0 bytes (empty)")
            return np.array([])

        df = pd.read_csv(path, header=None)
        if df.size == 0:
            print(f"[wesad_loader] WARNING: {path} has no values")
            return np.array([])
        return df.values.squeeze()

    except (EmptyDataError, ParserError) as e:
        print(f"[wesad_loader] WARNING: {path} raised {type(e).__name__}: {e}")
        return np.array([])


def load_subject_e4(subject_id: str,
                    base_path: str = WESAD_PATH) -> Dict[str, Any]:
    """
    Load Empatica E4 wrist signals for one subject.
    """
    subj_dir = os.path.join(base_path, subject_id)
    e4_dir = os.path.join(subj_dir, f"{subject_id}_E4_Data")

    if not os.path.isdir(e4_dir):
        raise FileNotFoundError(f"E4 folder not found for {subject_id} at {e4_dir}")

    data: Dict[str, Any] = {}

    for name in E4_FILES:
        csv_path = os.path.join(e4_dir, f"{name}.csv")
        if os.path.exists(csv_path):
            data[name.lower()] = _read_signal_csv(csv_path)
        else:
            print(f"[wesad_loader] WARNING: {csv_path} not found")
            data[name.lower()] = np.array([])

    # Parse metadata from info.txt if present
    info_path = os.path.join(e4_dir, "info.txt")
    meta: Dict[str, str] = {}
    if os.path.exists(info_path):
        with open(info_path, "r") as f:
            for line in f:
                line = line.strip()
                if not line or ":" not in line:
                    continue
                key, val = [x.strip() for x in line.split(":", 1)]
                meta[key] = val
    data["meta"] = meta

    return data


def list_subjects(base_path: str = WESAD_PATH) -> List[str]:
    """
    List all subject folders like 'S2', 'S3', ...
    """
    if not os.path.isdir(base_path):
        raise FileNotFoundError(f"WESAD base path not found: {base_path}")
    return sorted(
        d for d in os.listdir(base_path)
        if d.startswith("S") and os.path.isdir(os.path.join(base_path, d))
    )


# 1.4 Importing loader & list subjects
---



In [ ]:
import importlib
import wesad_loader
importlib.reload(wesad_loader)

from wesad_loader import load_subject_e4, list_subjects

print(os.listdir())  # sanity
print(list_subjects("data/WESAD"))

s2 = load_subject_e4("S2", base_path="data/WESAD")
for k, v in s2.items():
    if k == "meta":
        print(k, v)
    else:
        print(k, type(v), getattr(v, "shape", None))


The dataset was way too large to add to the github, so a way to work around this for reproducibility purposes is to essentially just use your own Kaggle API JSON so u can pull from their database

Will go into more detail later, but for now

1. Go to Kaggle settings
2. Create API token
3. Manually bind their own kaggle.JSON if needed
4. Upload it the same way to Colab when promted in the cell above.


2ndary Dataset I used:

Secondary dataset WESAD is 3GB+ and cannot be uploaded to colab or pushed to the git due to such a large file size, so in order to keep this reproducible, I am going to keep this here so that the team will be able to retrieve the data at runtime from the kaggle servers.

In [ ]:
def plot_wesad_eda_example(subject_data, n_seconds=300, fs=4):
    """
    Plot a snippet of EDA signal for one WESAD subject.
    - subject_data: dict returned by load_subject_e4(...)
    - n_seconds: length of snippet to show
    - fs: sampling frequency for EDA (Hz). WESAD EDA is usually 4 Hz.
    """
    eda = subject_data.get("eda", None)
    if eda is None or eda.size == 0:
        print("No EDA data found.")
        return

    n_samples = min(len(eda), n_seconds * fs)
    eda_snippet = eda[:n_samples]

    # Build time axis in seconds
    t = np.arange(n_samples) / fs

    plt.figure(figsize=(10, 4))
    plt.plot(t, eda_snippet)
    plt.xlabel("Time (s)")
    plt.ylabel("EDA (µS)")
    plt.title(f"WESAD: EDA snippet (first {n_seconds} seconds)")
    plt.tight_layout()
    plt.show()

plot_wesad_eda_example(s2, n_seconds=300, fs=4)


To understand physiological arousal, we first visualized raw EDA from WESAD. This plot shows 300 seconds of continuous skin-conductance from one subject. The fluctuations show moment-to-moment arousal changes something the AE dataset is missing. This justified using WESAD as a complementary source to learn general arousal patterns.

# 2. Affective Economics Dataset
---


In [ ]:
import pandas as pd
import re

print("Current directory:", os.getcwd())
print("Repo contents:", os.listdir())
print("DATASET contents:", os.listdir("DATASET"))

df = pd.read_csv("DATASET/AE_investment_dataset.csv")
df.head()
df.info()
df.isna().mean().sort_values().head(20)
df.columns.tolist()
for c in df.columns:
    print(c)





##Quick note bc I didn't know what PANAS meant:
- PANAS refers to the Positive and Negative Affect Schedule, a widely used psychological scale that measures an individual's mood by assessing both positive and negative emotions. Developed in 1988 by Watson, Clark, and Tellegen, it's a 20-item self-report measure used in research and clinical settings to gauge how frequently someone experiences emotions like interest, joy, enthusiasm (positive affect) versus feelings of distress, sadness, and nervousness (negative affect).
## How it works
- 20 items: The scale consists of 20 words that describe different feelings and emotions.
- Two dimensions: These items are separated into two subscales: one for positive affect (PA) and one for negative affect (NA).
- Rating scale: Participants rate how they felt about each item over a specific time frame (e.g., "right now," "today," "over the past few weeks") on a 5-point scale.
- Scoring: Each positive and negative item is scored individually. The total positive score and total negative score are then calculated. A higher positive score indicates more positive affect, while a higher negative score indicates more negative affect.

Building a per-trial table with:
1. inputs per step:
  - money_in_stocks
  - mean_return
  - stock_fluctuation
  - scr_anticipatory
2. Static inputs:
3. Target
  - Whether they invested in the stock (money_in_stocks > 0 -> 1 else 0)

#Kaggle Dataset for WESAD

-- note: ref doc later

# Minimal WESAD pipeline
- loading dataset, segmenting data into windows, computing basics, labelling windows as baselines vs stress.
- using dataframe and constructing just like we do later on for the Bath dataset

## 2.1 Loading Affective Economics Dataset
---

In [ ]:
import re
import pandas as pd

print("Current directory:", os.getcwd())
print("Repo contents:", os.listdir())
print("DATASET contents:", os.listdir("DATASET"))

df = pd.read_csv("DATASET/AE_investment_dataset.csv")
df.info()

# 0. ID & static columns
id_cols = ["Participant_code", "Age", "Gender", "Nationality", "Ethnicity", "Played_stock_market"]

# 1. Grab trial-level columns by prefix
stock_cols = [c for c in df.columns if c.startswith("Money_in_stocks_S")]
scr_cols   = [c for c in df.columns if c.startswith("SCR_AnticipatoryS")]
ret_cols   = [c for c in df.columns if c.startswith("Mean_Return_S")]
fluc_cols  = [c for c in df.columns if c.startswith("stock_fluctuation_S")]

print("n_stock_cols:", len(stock_cols))
print("n_scr_cols:", len(scr_cols))
print("n_return_cols:", len(ret_cols))
print("n_fluctuation_cols:", len(fluc_cols))

# 2. Map (session, trial) -> column name
def build_lookup(cols, prefix):
    lookup = {}
    for c in cols:
        # e.g. Money_in_stocks_S1_T3  →  session=1, trial=3
        m = re.match(rf"{re.escape(prefix)}(\d+)_T(\d+)$", c)
        if m:
            s = int(m.group(1))   # session number
            t = int(m.group(2))   # trial number within session
            lookup[(s, t)] = c
    return lookup

stock_map = build_lookup(stock_cols, "Money_in_stocks_S")
scr_map   = build_lookup(scr_cols,   "SCR_AnticipatoryS")
ret_map   = build_lookup(ret_cols,   "Mean_Return_S")
fluc_map  = build_lookup(fluc_cols,  "stock_fluctuation_S")

print("number of keys in stock_map:", len(stock_map))
print("some keys from stock_map:", list(stock_map.items())[:5])


## 2.2 Long format trial table

In [ ]:
rows = []

for _, row in df.iterrows():
    # carry participant-level info
    base = {col: row[col] for col in id_cols}

    for (s, t) in sorted(stock_map.keys()):
        rec = dict(base)
        rec["session"] = s
        rec["trial_in_session"] = t
        rec["global_trial"] = (s - 1) * 10 + t  # 1..40

        rec["money_in_stocks"] = row[stock_map[(s, t)]]
        rec["scr_anticipatory"] = row[scr_map[(s, t)]]

        # Some (session, trial) combos might not have return/fluctuation
        rec["mean_return"] = row[ret_map[(s, t)]] if (s, t) in ret_map else pd.NA
        rec["stock_fluctuation"] = row[fluc_map[(s, t)]] if (s, t) in fluc_map else pd.NA

        rows.append(rec)

long_df = pd.DataFrame(rows)

# Target: did they invest at all?
long_df["invested"] = (long_df["money_in_stocks"] > 0).astype(int)

print(long_df.shape)
long_df.head()
print(long_df["invested"].value_counts())


## 2.3 Cleaning + Basic Features


In [1]:


import pandas as pd
import numpy as np

# Drop rows where our key features are missing
key_features = ["scr_anticipatory", "mean_return", "stock_fluctuation", "money_in_stocks"]

clean_df = long_df.dropna(subset=key_features).copy()

# Convert types to numeric
for col in key_features:
    clean_df[col] = pd.to_numeric(clean_df[col], errors='coerce')

# Drop again if any become NA
clean_df = clean_df.dropna(subset=key_features)

# Target
y = clean_df["invested"].astype(int)

# Features: minimal baseline
X = clean_df[["scr_anticipatory", "mean_return", "stock_fluctuation"]]

print("Clean shape:", clean_df.shape)
X.head()


NameError: name 'long_df' is not defined

##2.4 Ablation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

def train_eval_logreg(X, y, desc):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred)

    print(f"\n=== {desc} ===")
    print("Accuracy:", acc)
    print("F1-score:", f1)
    print(classification_report(y_test, y_pred))

    return acc, f1

y = clean_df["invested"].astype(int)

# Physiology-only
X_phys = clean_df[["scr_anticipatory"]]

# Market-only
X_market = clean_df[["mean_return", "stock_fluctuation"]]

# Combined
X_both = clean_df[["scr_anticipatory", "mean_return", "stock_fluctuation"]]

acc_phys, f1_phys = train_eval_logreg(X_phys, y, "Physiology only (SCR)")
acc_mark, f1_mark = train_eval_logreg(X_market, y, "Market context only")
acc_both, f1_both = train_eval_logreg(X_both, y, "Physiology + Market")


 ### Correlation Heatmap of Models
 What it shows:
SCR positively correlated with investing
Returns/volatility relationships
Good for “Feature Analysis” slide

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr = clean_df[['scr_anticipatory','mean_return','stock_fluctuation','invested']].corr()

plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("AE Dataset — Correlation Heatmap")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Baseline model
clf = LogisticRegression()
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.neural_network import MLPClassifier

# Scale features again (safe to reuse)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# MLP model: small but strong
mlp = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)

y_pred_mlp = mlp.predict(X_test_scaled)

print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("MLP F1:", f1_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))


MLP Cell


In [ ]:
from sklearn.neural_network import MLPClassifier

#scale data again
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# MLP model: small but strong
mlp = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)

y_pred_mlp = mlp.predict(X_test_scaled)

print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("MLP F1:", f1_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))


In [ ]:
import numpy as np

# Work on a copy, sorted by participant + time
scr_df = clean_df.sort_values(["Participant_code", "session", "trial_in_session"]).copy()

grp = scr_df.groupby("Participant_code")

# Participant-level mean/std and z-score
scr_df["scr_mean_p"] = grp["scr_anticipatory"].transform("mean")
scr_df["scr_std_p"]  = grp["scr_anticipatory"].transform("std")
scr_df["scr_z"] = (scr_df["scr_anticipatory"] - scr_df["scr_mean_p"]) / scr_df["scr_std_p"]

# Lags within each participant
scr_df["scr_lag1"] = grp["scr_anticipatory"].shift(1)
scr_df["scr_lag2"] = grp["scr_anticipatory"].shift(2)

# Changes vs previous trials
scr_df["scr_delta1"] = scr_df["scr_anticipatory"] - scr_df["scr_lag1"]
scr_df["scr_delta2"] = scr_df["scr_anticipatory"] - scr_df["scr_lag2"]

# Short-term rolling window stats (window=3 trials)
scr_df["scr_roll_mean3"] = grp["scr_anticipatory"].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)
scr_df["scr_roll_std3"] = grp["scr_anticipatory"].transform(
    lambda x: x.rolling(window=3, min_periods=1).std()
)

# Replace NaNs from lags / std=0 with 0 for now
scr_feature_cols = [
    "scr_anticipatory",
    "scr_z",
    "scr_lag1", "scr_lag2",
    "scr_delta1", "scr_delta2",
    "scr_roll_mean3", "scr_roll_std3",
]

scr_df[scr_feature_cols] = scr_df[scr_feature_cols].fillna(0.0)

print("SCR feature shape:", scr_df[scr_feature_cols].shape)
scr_df[scr_feature_cols + ["invested"]].head()


## 3.3 Plotting AE: SCR + investing over trials for one participant


In [ ]:
def plot_ae_scr_timeseries(scr_df, participant_code):
    """
    Plot anticipatory SCR across trials for one participant,
    with invest vs not-invest marked.
    """
    # Filter and sort in time order
    subj = (
        scr_df[scr_df["Participant_code"] == participant_code]
        .sort_values(["session", "trial_in_session"])
        .copy()
    )
    if subj.empty:
        print(f"No data for Participant_code={participant_code}")
        return

    # Build a global index (1..N) just for plotting
    subj["trial_idx"] = range(1, len(subj) + 1)

    plt.figure(figsize=(10, 4))
    # Line of SCR
    plt.plot(subj["trial_idx"], subj["scr_anticipatory"], marker="o", label="SCR")

    # Mark invest vs not-invest
    invested = subj["invested"] == 1
    not_invested = subj["invested"] == 0

    plt.scatter(
        subj.loc[invested, "trial_idx"],
        subj.loc[invested, "scr_anticipatory"],
        marker="^",
        label="Invested (1)",
    )
    plt.scatter(
        subj.loc[not_invested, "trial_idx"],
        subj.loc[not_invested, "scr_anticipatory"],
        marker="x",
        label="Not invested (0)",
    )

    plt.xlabel("Trial (time)")
    plt.ylabel("Anticipatory SCR")
    plt.title(f"AE: SCR and investment decisions over trials (Participant {participant_code})")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# Pick one example participant from the data
example_pid = scr_df["Participant_code"].iloc[0]
plot_ae_scr_timeseries(scr_df, example_pid)


This figure shows anticipatory SCR (emotion/arousal) for a single AE participant plotted across the 40 investment trials. We annotated trials where the participant invested versus not. We observed that spikes in SCR often precede investment events, consistent with findings from affective neuroscience: anticipatory arousal tracks risky decision engagement.

## 3.4 Plotting AE with Market Context (Mean Return & Volatility) over trials

In [ ]:
def plot_ae_market_timeseries(scr_df, participant_code):
    """
    Plot mean return and stock fluctuation over trials for one participant.
    """
    subj = (
        scr_df[scr_df["Participant_code"] == participant_code]
        .sort_values(["session", "trial_in_session"])
        .copy()
    )
    if subj.empty:
        print(f"No data for Participant_code={participant_code}")
        return

    subj["trial_idx"] = range(1, len(subj) + 1)

    plt.figure(figsize=(10, 4))
    plt.plot(
        subj["trial_idx"], subj["mean_return"],
        marker="o", label="Mean return"
    )
    plt.plot(
        subj["trial_idx"], subj["stock_fluctuation"],
        marker="s", label="Stock fluctuation"
    )

    plt.xlabel("Trial (time)")
    plt.ylabel("Market values")
    plt.title(f"AE: Market context over trials (Participant {participant_code})")
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_ae_market_timeseries(scr_df, example_pid)


In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# ==== Prepare data ====
X_scr = scr_df[scr_feature_cols].values.astype("float32")
y_scr = scr_df["invested"].values.astype("int64")

X_train, X_val, y_train, y_val = train_test_split(
    X_scr, y_scr, test_size=0.2, random_state=42, stratify=y_scr
)

# Standardize SCR features
scaler_scr = StandardScaler()
X_train_scaled = scaler_scr.fit_transform(X_train)
X_val_scaled   = scaler_scr.transform(X_val)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor   = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor   = torch.tensor(y_val, dtype=torch.long)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
val_ds   = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)

input_dim = X_train_tensor.shape[1]
emb_dim   = 16

# ==== Physiology encoder ====
class PhysioEncoder(nn.Module):
    def __init__(self, in_dim, hidden_dim=32, emb_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, emb_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

# Full model: encoder + classifier head
class PhysioModel(nn.Module):
    def __init__(self, encoder, emb_dim):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(emb_dim, 2)

    def forward(self, x):
        z = self.encoder(x)        # [batch, emb_dim]
        logits = self.classifier(z)  # [batch, 2]
        return logits

encoder = PhysioEncoder(input_dim, hidden_dim=32, emb_dim=emb_dim)
model = PhysioModel(encoder, emb_dim)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ==== Training loop ====
n_epochs = 30

for epoch in range(1, n_epochs + 1):
    model.train()
    train_losses = []

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    # Validation
    model.eval()
    all_preds = []
    all_true  = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds.cpu())
            all_true.append(yb.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_true  = torch.cat(all_true).numpy()

    acc = accuracy_score(all_true, all_preds)
    f1  = f1_score(all_true, all_preds)

    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} | "
              f"train_loss={np.mean(train_losses):.4f} | "
              f"val_acc={acc:.3f} | val_f1={f1:.3f}")


#4.1 Evaluate

evaluation helper


In [ ]:
def evaluate_physio_model(model, X_scaled, y_true):
    model.eval()
    with torch.no_grad():
        logits = model(torch.tensor(X_scaled, dtype=torch.float32).to(device))
        preds = torch.argmax(logits, dim=1).cpu().numpy()

    acc = accuracy_score(y_true, preds)
    f1  = f1_score(y_true, preds)
    print("PhysioEncoder Accuracy:", acc)
    print("PhysioEncoder F1:", f1)
    return acc, f1

physio_acc, physio_f1 = evaluate_physio_model(model, X_val_scaled, y_val)


##4.1 Feature Pipeline and Classifer

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

def make_eda_windows(eda, window_sec=60, step_sec=30, fs=4):
    """
    Convert 1D EDA signal into overlapping windows.
    - window_sec: window length in seconds
    - step_sec: step size between windows in seconds
    - fs: sampling rate (Hz) for EDA (WESAD E4 EDA = 4 Hz)
    """
    # EDA in WESAD E4 has first two rows = [start_time, sampling_rate]
    # Strip them out:
    if len(eda) > 2:
        eda_values = eda[2:].astype(float)
    else:
        return np.empty((0, 5)), np.empty((0,), dtype=int)

    win_len = window_sec * fs
    step_len = step_sec * fs

    windows = []
    for start in range(0, len(eda_values) - win_len, step_len):
        seg = eda_values[start:start + win_len]
        if len(seg) < win_len:
            break
        # basic stats
        mean = seg.mean()
        std  = seg.std()
        max_ = seg.max()
        min_ = seg.min()
        slope = (seg[-1] - seg[0]) / window_sec  # change per second
        windows.append([mean, std, max_, min_, slope])

    if not windows:
        return np.empty((0, 5)), np.empty((0,), dtype=int)

    windows = np.array(windows)

    # Proxy high/low arousal via z-score of mean EDA across windows
    z = (windows[:, 0] - windows[:, 0].mean()) / (windows[:, 0].std() + 1e-8)
    labels = (z > 0.5).astype(int)  # 1 = “higher arousal”, 0 = “lower arousal”

    return windows, labels


## 4.3 Dataset Across multiple subjects

In [ ]:
all_X = []
all_y = []

subjects = list_subjects("data/WESAD")
print("Subjects:", subjects)

for sid in subjects:
    s = load_subject_e4(sid, base_path="data/WESAD")
    eda = s["eda"]
    X_win, y_win = make_eda_windows(eda)
    if len(X_win) == 0:
        continue
    all_X.append(X_win)
    all_y.append(y_win)

X_eda = np.vstack(all_X)
y_eda = np.concatenate(all_y)

print("WESAD windowed EDA shape:", X_eda.shape, "labels:", np.bincount(y_eda))


## 4.4 Classifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(
    X_eda, y_eda, test_size=0.2, random_state=42, stratify=y_eda
)

scaler_w = StandardScaler()
X_train_w_scaled = scaler_w.fit_transform(X_train_w)
X_test_w_scaled  = scaler_w.transform(X_test_w)

clf_w = LogisticRegression(max_iter=1000)
clf_w.fit(X_train_w_scaled, y_train_w)
y_pred_w = clf_w.predict(X_test_w_scaled)

print("WESAD EDA baseline Accuracy:", accuracy_score(y_test_w, y_pred_w))
print("WESAD EDA baseline F1:", f1_score(y_test_w, y_pred_w))
print(classification_report(y_test_w, y_pred_w))


AE: we model trial-level anticipatory SCR + market context → risky vs safe choice
WESAD: we model time-windowed continuous EDA → low vs high arousal (proxy labels)
Both pipelines use small neural models on physiological features.
This justifies the phrase “multimodal physiological representation learning”.
Is the WESAD label scheme “perfect”? No. But for a course project, if you’re explicit (“weak supervision via EDA z-score as arousal proxy”), it’s absolutely fine.

#Results & Write-up

In [ ]:
def plot_investment_class_distribution(clean_df):
    counts = clean_df["invested"].value_counts().sort_index()
    plt.figure(figsize=(4, 3))
    counts.plot(kind="bar")
    plt.xticks([0, 1], ["Not invested (0)", "Invested (1)"], rotation=0)
    plt.ylabel("Number of trials")
    plt.title("Class distribution of investment decisions")
    plt.tight_layout()
    plt.show()
    print(counts)

plot_investment_class_distribution(clean_df)


  ### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred_mlp)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Invest", "Invest"])
disp.plot(cmap="Blues")
plt.title("MLP Confusion Matrix")
plt.show()


Ablation Plots (Market-only vs SCR-only vs both)

In [ ]:
# MARKET-ONLY FEATURES
X_mkt = clean_df[['mean_return','stock_fluctuation']]
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_mkt, y, test_size=0.2, random_state=42, stratify=y)

scaler_m = StandardScaler()
X_train_m = scaler_m.fit_transform(X_train_m)
X_test_m  = scaler_m.transform(X_test_m)

log_m = LogisticRegression().fit(X_train_m, y_train_m)
acc_market = accuracy_score(y_test_m, log_m.predict(X_test_m))

# SCR-ONLY FEATURES
X_scr_only = clean_df[['scr_anticipatory']]
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_scr_only, y, test_size=0.2, random_state=42, stratify=y)

scaler_s = StandardScaler()
X_train_s = scaler_s.fit_transform(X_train_s)
X_test_s  = scaler_s.transform(X_test_s)

log_s = LogisticRegression().fit(X_train_s, y_train_s)
acc_scr = accuracy_score(y_test_s, log_s.predict(X_test_s))

# COMBINED (already done)
acc_combined = accuracy_score(y_test, y_pred)

#now plot it
plt.figure(figsize=(6,4))
plt.bar(["Market only", "SCR only", "Combined"], [acc_market, acc_scr, acc_combined],
        color=["#6baed6", "#74c476", "#fd8d3c"])
plt.ylabel("Accuracy")
plt.title("Ablation Study — Contribution of Each Feature Set")
plt.tight_layout()
plt.show()



MLP Loss Curve

In [ ]:
train_losses = []
val_accs = []
val_f1s = []

for epoch in range(1, n_epochs + 1):
    model.train()
    batch_losses = []
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        batch_losses.append(loss.item())
    train_losses.append(np.mean(batch_losses))

    # validation
    model.eval()
    all_preds, all_true = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds.cpu())
            all_true.append(yb.cpu())
    all_preds = torch.cat(all_preds).numpy()
    all_true  = torch.cat(all_true).numpy()
    val_accs.append(accuracy_score(all_true, all_preds))
    val_f1s.append(f1_score(all_true, all_preds))

    #plot it

    plt.figure(figsize=(6,4))
plt.plot(train_losses, label="Train Loss")
plt.title("MLP Training Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(6,4))
plt.plot(val_accs, label="Val Accuracy")
plt.plot(val_f1s, label="Val F1")
plt.title("MLP Validation Curves")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.show()



For further analysis, because we have only been plotting for one prticipant, here is SCR distribution across ALL participants.

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(clean_df["scr_anticipatory"], bins=30, color="#9ecae1", edgecolor="black")
plt.title("Distribution of SCR Across All Participants")
plt.xlabel("SCR (µS)")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


Here is Average SCR by participant

In [ ]:
avg_scr = scr_df.groupby("Participant_code")["scr_anticipatory"].mean()

plt.figure(figsize=(8,4))
avg_scr.plot(kind="bar", color="#74c476")
plt.title("Average SCR per Participant")
plt.xlabel("Participant ID")
plt.ylabel("Mean SCR")
plt.tight_layout()
plt.show()


Investment Rate by Participant

In [ ]:
inv = scr_df.groupby("Participant_code")["invested"].mean()

plt.figure(figsize=(8,4))
inv.plot(kind="bar", color="#fd8d3c")
plt.title("Investment Rate per Participant")
plt.xlabel("Participant ID")
plt.ylabel("% Invested")
plt.tight_layout()
plt.show()


Relationship Plot For SCR -> Probability of investment

In [ ]:
import seaborn as sns

plt.figure(figsize=(6,4))
sns.regplot(x=scr_df["scr_anticipatory"], y=scr_df["invested"],
            logistic=True, scatter_kws={"alpha":0.3})
plt.title("Logistic Relationship Between SCR and Investment Probability")
plt.xlabel("SCR (µS)")
plt.ylabel("P(Invest)")
plt.show()
